In [1]:
import pandas as pd
import base64
import os

from requests import get
from bs4 import BeautifulSoup
from time import sleep
## Create empty df:
# column_names = ["Artist", "Song", "Song_ID", "Lyrics", "Url"]
# df = pd.DataFrame(columns = column_names)

##OR##

# Load existing data and add to it:
df = pd.read_csv('data.csv')

,Artist,Song,Song_ID,Lyrics,Url
0,Novi fosili,Ako se rastanemo,oOIOCzZV_0,"ne govorim s tobom,\r\ni ništa nije kao prije,...",https://cuspajz.com//tekstovi-pjesama/pjesma/n...
1,Novi fosili,Bilo mi je prvi put,oOIOCzZV_1,Bilo je nebo na tvome čelu\r\nU srcu zdjela vr...,https://cuspajz.com//tekstovi-pjesama/pjesma/n...
2,Novi fosili,"Čuješ li me, je l' ti drago",oOIOCzZV_2,NaN,https://cuspajz.com//tekstovi-pjesama/pjesma/n...
3,Novi fosili,Dijete sreće,oOIOCzZV_3,ja sam dijete srece\r\ni nepostojim\r\nne voli...,https://cuspajz.com//tekstovi-pjesama/pjesma/n...
4,Novi fosili,Djeca ljubavi,oOIOCzZV_4,"Prvi put ti kazem da, da se bojim svitanja\r\n...",https://cuspajz.com//tekstovi-pjesama/pjesma/n...
...,...,...,...,...,...
283,Mate Miso Kovac,"Zbog tebe, ljubavi moja",Zwn2Yv0K_97,NaN,https://cuspajz.com//tekstovi-pjesama/pjesma/m...
284,Mate Miso Kovac,Zbogom zvijezdo,Zwn2Yv0K_98,Moja lađa neće više\r\nPristati u tvoju luku\r...,https://cuspajz.com//tekstovi-pjesama/pjesma/m...
285,Mate Miso Kovac,"Zeleno, zeleno",Zwn2Yv0K_99,Kao snijeg je bijelo\r\nKo kesten je smeđe\r\n...,https://cuspajz.com//tekstovi-pjesama/pjesma/m...
286,Mate Miso Kovac,Znam da si sama,Zwn2Yv0K_100,Znam da si sama\r\nZnam da te srce boli\r\nNe ...,https://cuspajz.com//tekstovi-pjesama/pjesma/m...


In [2]:
# Scraping cuspajz.com for song lyrics by artist (change url to change artist)

# Based on:
#    https://www.dataquest.io/blog/web-scraping-beautifulsoup/
#    https://github.com/aakashbansal/Songs-Lyrics-Web-Scraper/blob/master/Lyrics%20Scraper.ipynb

artist = 'Severina'
# artist url
url = 'https://cuspajz.com/tekstovi-pjesama/izvodjac/severina.html'

# generate random artist ID and make sure it doesnt have '_'
artist_ID = base64.b64encode(os.urandom(6)).decode('ascii')
while(artist_ID.find('_')>=0):
    artist_ID = base64.b64encode(os.urandom(6)).decode('ascii')
    
delay = 2

response = get(url)
# print(response.text[:500])
html_soup = BeautifulSoup(response.text, 'html.parser')
# print(type(html_soup))


# Get the list of songs
songs = html_soup.find_all('ul', class_='songList')



In [3]:

    for col_index, column in enumerate(songs):
        songs = column.find_all('li')

        for index, song in enumerate(songs):

            song_name = song.a.text
            song_url = song.a['href']

            url = 'https://cuspajz.com/'+song_url

            response = get(url)

            html_soup = BeautifulSoup(response.text, 'html.parser')

            song_lyrics = html_soup.find_all('p', class_='clearfix')
            lyrics =  song_lyrics[0].get_text()
            
            song_ID = artist_ID + '_' + str(1) + '_' + str(index)  

            new_entry = pd.DataFrame([[artist, song_name, song_ID, lyrics, url]], columns=["Artist", "Song", "Song_ID", "Lyrics", "Url"])
            df = df.append(new_entry, ignore_index=True)


#             file.write("###"+song_name+"###")
#             file.write("\n\n")
#             file.write(lyrics)
#             file.write("\n\n")

            print("Lyrics successfully written to file for : " + song_name)
                      
            sleep(delay)
                      

NameError: name 'df' is not defined

In [126]:
df.to_csv('data.csv', index=False)

'Zwn2Yv0K_1_101'

In [137]:
df[0:45]

,Artist,Song,Song_ID,Lyrics,Url
0,Novi fosili,Ako se rastanemo,oOIOCzZV_0,"ne govorim s tobom,\r\ni ništa nije kao prije,...",https://cuspajz.com//tekstovi-pjesama/pjesma/n...
1,Novi fosili,Bilo mi je prvi put,oOIOCzZV_1,Bilo je nebo na tvome čelu\r\nU srcu zdjela vr...,https://cuspajz.com//tekstovi-pjesama/pjesma/n...
2,Novi fosili,"Čuješ li me, je l' ti drago",oOIOCzZV_2,,https://cuspajz.com//tekstovi-pjesama/pjesma/n...
3,Novi fosili,Dijete sreće,oOIOCzZV_3,ja sam dijete srece\r\ni nepostojim\r\nne voli...,https://cuspajz.com//tekstovi-pjesama/pjesma/n...
4,Novi fosili,Djeca ljubavi,oOIOCzZV_4,"Prvi put ti kazem da, da se bojim svitanja\r\n...",https://cuspajz.com//tekstovi-pjesama/pjesma/n...
5,Novi fosili,Dobre djevojke,oOIOCzZV_5,"Budi dobra ti, nemoj gradom lutati\r\nmini ne ...",https://cuspajz.com//tekstovi-pjesama/pjesma/n...
6,Novi fosili,E da si barem noćas ovdje,oOIOCzZV_6,Obuci cu nocas haljinu sto volis\r\nDignuti ko...,https://cuspajz.com//tekstovi-pjesama/pjesma/n...
7,Novi fosili,Hoću li znati,oOIOCzZV_7,"Hoću li znati, hoću li\r\nTvoju ljubav zauvjek...",https://cuspajz.com//tekstovi-pjesama/pjesma/n...
8,Novi fosili,Ja sam za ples,oOIOCzZV_8,I wanna dance! O ho...\r\nJa sam za ples! O je...,https://cuspajz.com//tekstovi-pjesama/pjesma/n...
9,Novi fosili,Jedna suza,oOIOCzZV_9,"Jedna suza, jedna suza\n\r\nHej vlakovi zorom ...",https://cuspajz.com//tekstovi-pjesama/pjesma/n...
